# This code demonstrates how you can use monolingual and cross-lingual embeddings to train machine learning models for downstream tasks (Part of Speech Tagging or Named Entity Recognition). In this code you will observe the following:
* How to visualize monolingual and cross-lingual embeddings
* How to train a sequance to sequance model with cross-lingual and monolingual embeddings. In this example code we used Bidirectional Long Short Term Memory (BiLSTM), however you can do the same with RNN, LSTM, GRU, and BiGRU (as commented in the code)
* We start with Part of Speech Tagging.

In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from matplotlib import pyplot as plt
import seaborn as sns
from gensim.models import KeyedVectors
import tensorflow as tf
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding
from tensorflow.keras.layers import Dense, Input
from tensorflow.keras.layers import TimeDistributed
from tensorflow.keras.layers import LSTM, GRU, Bidirectional, SimpleRNN, RNN
from tensorflow.keras.models import Model
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow import keras
from tensorflow.keras.callbacks import Callback, LearningRateScheduler, ReduceLROnPlateau
from sklearn.metrics import precision_score, recall_score, f1_score
from sklearn.model_selection import train_test_split
from sklearn.utils import shuffle
import os
import sys
import nltk
import re
# Download the punkt tokenizer if not already downloaded
#nltk.download('punkt')
import random
from nltk.tokenize import sent_tokenize
from nltk.tokenize import word_tokenize
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import MultiLabelBinarizer
#from tf.keras.callbacks import LearningRateScheduler

In [ ]:
def step_decay_schedule(initial_lr=1e-3, decay_factor=0.75, step_size=10):
    '''
    Wrapper function to create a LearningRateScheduler with step decay schedule.
    '''
    def schedule(epoch):
        return initial_lr * (decay_factor ** np.floor(epoch/step_size))

    return LearningRateScheduler(schedule)

class F1ScoreCallback(Callback):
    def __init__(self,X_train, Y_train, X_val, Y_val):
        self.X_train = X_train
        self.Y_train = Y_train
        self.X_val = X_val
        self.Y_val = Y_val

    def on_epoch_end(self, epoch, logs=None):
        # Predict on training data
        y_train_pred = self.model.predict(self.X_train)
        y_train_pred_flat = y_train_pred.argmax(axis=-1).flatten()
        y_train_true_flat = self.Y_train.argmax(axis=-1).flatten()

        # Predict on validation data
        y_val_pred = self.model.predict(self.X_val)
        y_val_pred_flat = y_val_pred.argmax(axis=-1).flatten()
        y_val_true_flat = self.Y_val.argmax(axis=-1).flatten()

        # Calculate precision, recall, and F1 score for training data
        train_precision = precision_score(y_train_true_flat, y_train_pred_flat, average='weighted')
        train_recall = recall_score(y_train_true_flat, y_train_pred_flat, average='weighted')
        train_f1 = f1_score(y_train_true_flat, y_train_pred_flat, average='weighted')

        # Calculate precision, recall, and F1 score for validation data
        val_precision = precision_score(y_val_true_flat, y_val_pred_flat, average='weighted')
        val_recall = recall_score(y_val_true_flat, y_val_pred_flat, average='weighted')
        val_f1 = f1_score(y_val_true_flat, y_val_pred_flat, average='weighted')

        # Print metrics
        print(f'\nEpoch {epoch + 1} Metrics:')
        print(f'Training Precision: {train_precision:.4f}, Training Recall: {train_recall:.4f}, Training F1 Score: {train_f1:.4f}')
        print(f'Validation Precision: {val_precision:.4f}, Validation Recall: {val_recall:.4f}, Validation F1 Score: {val_f1:.4f}\n')

        # Add metrics to logs for TensorBoard or other monitoring tools
        logs['train_precision'] = train_precision
        logs['train_recall'] = train_recall
        logs['train_f1'] = train_f1
        logs['val_precision'] = val_precision
        logs['val_recall'] = val_recall
        logs['val_f1'] = val_f1



# In[4]:


def f1_rec_prec(passed_model, X_test , Y_test):
        Y_pred = passed_model.predict(X_test)
        # Convert predictions and ground truth to 1D arrays
        Y_pred_flat = Y_pred.argmax(axis=-1).flatten()
        Y_true_flat = Y_test.argmax(axis=-1).flatten()

        # Calculate precision, recall, and F1 score
        precision = precision_score(Y_true_flat, Y_pred_flat, average='weighted')
        recall = recall_score(Y_true_flat, Y_pred_flat, average='weighted')
        f1 = f1_score(Y_true_flat, Y_pred_flat, average='weighted')
        return f1, recall, precision

# inference
def sequence_to_tag(list_of_indices, reverse_word_map_param):
    # Looking up words in dictionary
    print(len(list_of_indices[0]))
    sentence_labels = [np.argmax(token_probs) for token_probs in list_of_indices]
    #pred_sequence_no_padding = [token for token in list_of_indices.argmax(axis=-1) if token != 0]
    words = [reverse_word_map_param[letter] for letter in sentence_labels if letter != 0]
    return(words)

def sequence_to_text(list_of_indices, reverse_word_map_param):
    # Looking up words in dictionary
    words = [reverse_word_map_param[letter] for letter in list(list_of_indices) if letter != 0]

    return(words)

def inference(passed_model,  X_test,  tokenizer_tag, tokenizer_words, filename ):
              Y_pred = passed_model.predict(X_test)
              # loop through sentences
              reverse_word_map = dict(map(reversed, tokenizer_tag.word_index.items()))
              print(len(reverse_word_map))
              rev_text_word_map = dict(map(reversed, tokenizer_words.word_index.items()))

              # write to file
              with open(filename, "w") as f_name:
                        for i, sent in enumerate(X_test):
                                      #s_split = sent.split()
                                      s_split  = sequence_to_text( sent, rev_text_word_map)
                                      print(s_split)
                                      predic_text = sequence_to_tag(Y_pred[i], reverse_word_map)
                                      print(predic_text)
                                      for p, pred in enumerate(predic_text):
                                                    try:
                                                            f_name.write(s_split[p] + " " + pred + "\n")
                                                    except:
                                                            pass
                                      f_name.write("\n")

# In[5]:
def confusion_m_plot(passed_model, X_test , Y_test, lang, label_tokenizer, m_name):
            Y_pred = passed_model.predict(X_test)
            # Convert predictions and ground truth to 1D arrays
            Y_pred_flat = Y_pred.argmax(axis=-1).flatten()
            Y_true_flat = Y_test.argmax(axis=-1).flatten()

            index_to_label = {index: label for label, index in label_tokenizer.word_index.items()}
            #decoded_label_Pred = [index for index in Y_pred_flat.argmax()]
            #print(decoded_label_Pred)
            #decoded_label_Y = [index for index in Y_true_flat.argmax()]


            label_orig = list(index_to_label.values())
            # Generate the confusion matrix
            conf_matrix = confusion_matrix(Y_true_flat, Y_pred_flat)
            conf_matrix = conf_matrix.astype('float') / conf_matrix.sum(axis=1)[:, np.newaxis]
            # Plot the confusion matrix using seaborn
            plt.figure(figsize=(8, 6))
            sns.heatmap(conf_matrix, annot=True, fmt='.2g', cmap='Blues', xticklabels=label_orig, yticklabels=label_orig)
            plt.title('Confusion Matrix')
            plt.xlabel('Predicted Labels')
            plt.ylabel('True Labels')
            plt.savefig(confus_path + m_name + lang + ".jpeg", format='jpeg', dpi=300)
#             plt.show()



class Data_Model_Wrapper_Code_Switch:
        def __init__(self,root_data, root_mono, root_cross, path_to_destination_eda):
                    # initialize paths
                    self.data_path     = root_data
                    self.path_to_mono_emb = root_mono + "/"
                    self.path_to_cross_emb = root_cross + "/"
                    self.path_to_eda_output = path_to_destination_eda + "/"

        # load embeddings
        def get_emb_mat(self, src_language, trg_language):
                    # loading cross_lingual embeddings
                    s_t_folder = src_language + "_" + trg_language
                    s_t_file_src = src_language + "_" + trg_language + "-src-vcmp.txt"
                    s_t_file_trg = src_language + "_" + trg_language + "-trg-vcmp.txt"

                    src_x_emb_file_p = self.path_to_cross_emb + s_t_folder + "/" + s_t_file_src
                    trg_x_emb_file_p = self.path_to_cross_emb + s_t_folder + "/" + s_t_file_trg

                    # load src cross emb
                    # src_embeddings_index = {}
                    # with open(src_x_emb_file_p) as f:
                    #                 for  line in f:
                    #                         word, coefs = line.split(maxsplit=1)
                    #                         coefs = np.fromstring(coefs, "f", sep=" ")
                    #                         src_embeddings_index[word] = coefs

                    # load target cross embeddings
                    trg_embeddings_index = {}
                    with open(trg_x_emb_file_p) as f:
                                    for  line in f:
                                            word, coefs = line.split(maxsplit=1)
                                            coefs = np.fromstring(coefs, "f", sep=" ")
                                            trg_embeddings_index[word] = coefs


                    src_vocab = self.word_tokenizer.word_index
                    self.src_num_tokens = len(self.word_tokenizer.word_index) + 1
                    #print(src_num_tokens)
                    embedding_dim =  200
                    hits_src = 0
                    misses_src = 0
                    hits_trg = 0
                    misses_trg = 0
                    input_size = 200
                    # self.code_switch_embedding_matrix = np.zeros((self.src_num_tokens, embedding_dim))
                    # for word, i in word2idx_all.items():
                    self.trg_embedding_matrix = np.zeros((self.src_num_tokens, embedding_dim))
                    # embeddings_index    =  self.code_switch_emb(src_embeddings_index, trg_embeddings_index)
                    for word in list(src_vocab.keys()):
                                    # trg emb
                                    embedding_vector = trg_embeddings_index.get(word)
                                    if embedding_vector is not None:
                                        self.trg_embedding_matrix[src_vocab[word]] = embedding_vector
                                        # self.code_switch_embedding_matrix[src_vocab[word]] = embedding_vector
                                        hits_trg +=1
                                    else:

                                        misses_trg += 1

    #                   return code_switch_layer

        # load data
        def load_train_dev_test(self):
                            # load train csv and devide data into train, dev and test.
                            try:
                                train = open(os.path.join(self.data_path ,'train.txt'),'r').readlines()
                            except FileNotFoundError:
                                train = ''
                            try:
                                dev = open(os.path.join(self.data_path ,'dev.txt'),'r').readlines()
                            except FileNotFoundError:
                                dev = ''

                            try:
                                test = open(os.path.join(self.data_path ,'test.txt'),'r').readlines()
                            except FileNotFoundError:
                                test = ''

                            new = []
                            sent = []
                            tag = []
                            X = []
                            Y = []
                            for t in train:
                                if t.strip() != '':
                                    split = t.strip().split(' ')
                                    if len(split) > 1:
                                        #print(split)
                                        sent.append(split[0])
                                        tag.append(split[1])

                                else:
                                    new_s = ' '.join(sent)
                                    new_t = ' '.join(tag)
                                    X.append(sent)
                                    Y.append(tag)
                                    new.append([new_s,new_t])
                                    sent = []
                                    tag = []

                            sent = []
                            tag = []
                            x_l = len(X)

                            if dev != '':
                                for d in dev:
                                        if d.strip() != '':
                                            split = d.strip().split(' ')
                                            sent.append(split[0])
                                            tag.append(split[1])
                                        else:
                                            new_s = ' '.join(sent)
                                            new_t = ' '.join(tag)
                                            X.append(sent)
                                            Y.append(tag)
                                            new.append([new_s,new_t])
                                            sent = []
                                            tag = []

                            d_l = len(X) - x_l

                            sent = []
                            tag = []

                            if test != '':
                                for tt in test:
                                    if tt.strip() != '':
                                        split = tt.strip().split(' ')
                                        sent.append(split[0])
                                        tag.append(split[1])
                                    else:
                                        new_s = ' '.join(sent)
                                        new_t = ' '.join(tag)
                                        X.append(sent)
                                        Y.append(tag)
                                        new.append([new_s,new_t])
                                        sent = []
                                        tag = []
                            self.X_text        =  [" ".join(x) for x in X[x_l + d_l:]]


                            num_words = len(set([word.lower() for sentence in X for word in sentence]))
                            num_tags   = len(set([word.lower() for sentence in Y for word in sentence]))
                            print("Total number of tagged sentences: {}".format(len(X)))
                            print("Vocabulary size: {}".format(num_words))
                            print("Total number of tags: {}".format(num_tags))



                            self.word_tokenizer = Tokenizer()                      # instantiate tokeniser
                            self.word_tokenizer.fit_on_texts(X)                    # fit tokeniser on data
                            X_encoded = self.word_tokenizer.texts_to_sequences(X)  # use the tokeniser to encode input sequence

                            # encode Y
                            self.tag_tokenizer = Tokenizer()
                            self.tag_tokenizer.fit_on_texts(Y)
                            Y_encoded = self.tag_tokenizer.texts_to_sequences(Y)

                            # make sure that each sequence of input and output is same length
                            different_length = [1 if len(input) != len(output) else 0 for input, output in zip(X_encoded, Y_encoded)]
                            print("{} sentences have disparate input-output lengths.".format(sum(different_length)))

                            lengths = [len(seq) for seq in X_encoded]
                            self.MAX_SEQ_LENGTH = max(lengths)  # sequences greater than 100 in length will be truncated
                            X_padded = pad_sequences(X_encoded, maxlen=self.MAX_SEQ_LENGTH, padding="pre", truncating="post")
                            Y_padded = pad_sequences(Y_encoded, maxlen=self.MAX_SEQ_LENGTH, padding="pre", truncating="post")

                            # assign padded sequences to X and Y
                            X, Y = X_padded, Y_padded
                            Y = to_categorical(Y)
                            Y = [array[:, 1:] for array in Y]
                            # Convert the result back to a list of NumPy arrays
                            Y = np.array([np.array(arr) for arr in Y])


                            self.X_train       =  X[:x_l]
                            self.Y_train       =  Y[:x_l]
                            self.X_test        =  X[x_l + d_l: ]
                            self.Y_test        =  Y[x_l + d_l: ]

                            self.X_validation  =   X[x_l: x_l + d_l]
                            self.Y_validation  =   Y[x_l: x_l + d_l]


                            # print number of samples in each set
                            print("TRAINING DATA")
                            print('Shape of input sequences: {}'.format(self.X_train.shape))
                            print('Shape of output sequences: {}'.format(self.Y_train.shape))
                            print("-"*50)
                            print("VALIDATION DATA")
                            print('Shape of input sequences: {}'.format(self.X_validation.shape))
                            print('Shape of output sequences: {}'.format(self.Y_validation.shape))
                            print("-"*50)
                            print("TESTING DATA")
                            print('Shape of input sequences: {}'.format(self.X_test.shape))
                            print('Shape of output sequences: {}'.format(self.Y_test.shape))
                            self.NUM_CLASSES = Y.shape[2]
                            #print(Y.shape)
    #                       return X_train, Y_train, X_validation, Y_validation, X_test,Y_test


        # training models
        def seq_to_seq_models(self,epochs, batch_s, type, destination, embed_type, source, target_lang, EMBEDDING_SIZE):

                            # Assuming you have self.X_validation and self.Y_validation for validation data
                            f1_callback = F1ScoreCallback(self.X_train, self.Y_train, self.X_validation, self.Y_validation)
                            '''
                            lr_sched_rn = step_decay_schedule(initial_lr=1e-3, decay_factor=0.45, step_size=4)
                            # Define a callback to modify the learning rate dynamically
                            earl_rnn = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=4)

                            # create RNN architecture
                            opt_rnn = tf.keras.optimizers.Adam()
                            self.rnn_model = Sequential()
                            # create embedding layer - usually the first layer in text problems
                            self.rnn_model.add(Embedding(input_dim   =  self.src_num_tokens,         # vocabulary size - number of unique words in data
                            output_dim    =  EMBEDDING_SIZE,          # length of vector with which each word is represented
                            input_length  =  self.MAX_SEQ_LENGTH,          # length of input sequence
                            weights       = [self.code_switch_embedding_matrix],      # word embedding matrix
                            trainable     =  True                     # True - update the embeddings while training
                            ))
                            # add an RNN layer which contains 64 RNN cells
                            self.rnn_model.add(SimpleRNN(64,
                                        return_sequences=True  # True - return whole sequence; False - return single output of the end of the sequence
                            ))

                            # add time distributed (output at each sequence) layer
                            self.rnn_model.add(Dense(32, activation='relu'))
                            self.rnn_model.add(TimeDistributed(Dense(self.NUM_CLASSES, activation='softmax')))
                            self.rnn_model.compile(loss      =  'categorical_crossentropy',
                                            optimizer =  opt_rnn,
                                            metrics   =  ['acc'])
                            print('\n','************* TRAINING RNN *************','\n')
                            rnn_training = self.rnn_model.fit(self.X_train, self.Y_train, batch_size=batch_s, epochs=epochs, validation_data=(self.X_validation, self.Y_validation),callbacks=[f1_callback, lr_sched_rn, earl_rnn], verbose = 2)
                            img_rnn =  img_dir + "X_rnn_acc_plot_" +source + '-' + target_lang + ".eps"
                            # visualise training history
                            plt.plot(rnn_training.history['acc'])
                            plt.plot(rnn_training.history['val_acc'])
                            plt.title('model accuracy')
                            plt.ylabel('accuracy')
                            plt.xlabel('epoch')
                            #plt.legend(['rnn_train', 'rnn_test'], loc="lower right")
                            plt.savefig(img_rnn, format='eps')
                            plt.show()
                            '''
                            '''
                            lr_sched_lstm = step_decay_schedule(initial_lr=1e-3, decay_factor=0.6, step_size=8)
                            earl_lstm = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=4)
                            # create LSTM architecture10
                            #opt_lstm = tf.keras.optimizers.Adam(learning_rate=0.0001)
                            opt_lstm = tf.keras.optimizers.Adam()
                            self.lstm_model = Sequential()
                            self.lstm_model.add(Embedding(input_dim   =  self.src_num_tokens,         # vocabulary size - number of unique words in data
                            output_dim    =  EMBEDDING_SIZE,          # length of vector with which each word is represented
                            input_length  =  self.MAX_SEQ_LENGTH,          # length of input sequence
                            weights       = [self.code_switch_embedding_matrix],      # word embedding matrix
                            trainable     =  True                     # True - update the embeddings while training
                            ))
                            self.lstm_model.add(LSTM(64, return_sequences=True))
                            self.lstm_model.add(Dense(32, activation='relu'))
                            self.lstm_model.add(TimeDistributed(Dense(self.NUM_CLASSES, activation='softmax')))
                            self.lstm_model.compile(loss      =  'categorical_crossentropy',
                                            optimizer =  opt_lstm,
                                            metrics   =  ['acc'])
                            print('\n','************* TRAINING LSTM *************','\n')
                            lstm_training = self.lstm_model.fit(self.X_train, self.Y_train, batch_size=batch_s, epochs=epochs, validation_data=(self.X_validation, self.Y_validation),callbacks=[f1_callback,lr_sched_lstm, earl_lstm], verbose = 2)
                            img_lstm = img_dir + "X_lstm_acc_plot_" +source + '-' + target_lang + ".eps"
                            # visualise training history
                            plt.plot(lstm_training.history['acc'])
                            plt.plot(lstm_training.history['val_acc'])
                            plt.title('model accuracy')
                            plt.ylabel('accuracy')
                            plt.xlabel('epoch')
                            #plt.legend(['lstm_train', 'lstm_test'], loc="lower right")
                            plt.savefig(img_lstm, format='eps')
                            plt.show()


                            # create GRU architecture
                            lr_sched_gru = step_decay_schedule(initial_lr=1e-2, decay_factor=0.5, step_size=4)
                            earl_gru = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=4)
                            #opt_gru = tf.keras.optimizers.Adam(learning_rate=0.001)
                            opt_gru = tf.keras.optimizers.Adam()
                            self.gru_model = Sequential()
                            self.gru_model.add(Embedding(input_dim   =  self.src_num_tokens,         # vocabulary size - number of unique words in data
                            output_dim    =  EMBEDDING_SIZE,          # length of vector with which each word is represented
                            input_length  =  self.MAX_SEQ_LENGTH,          # length of input sequence
                            weights       = [self.code_switch_embedding_matrix],      # word embedding matrix
                            trainable     =  True                     # True - update the embeddings while training
                            ))
                            self.gru_model.add(GRU(64, return_sequences=True))
                            self.gru_model.add(Dense(32, activation='relu'))
                            self.gru_model.add(TimeDistributed(Dense(self.NUM_CLASSES, activation='softmax')))
                            self.gru_model.compile(loss='categorical_crossentropy',
                                        optimizer= opt_gru,
                                        metrics=['acc'])
                            print('\n','************* TRAINING GRU *************','\n')
                            gru_training = self.gru_model.fit(self.X_train, self.Y_train, batch_size=batch_s, epochs=epochs, validation_data=(self.X_validation, self.Y_validation), callbacks=[f1_callback, lr_sched_gru, earl_gru], verbose = 2 )
                            img_gru = img_dir + "X_gru_acc_plot_" +source + '-' + target_lang + ".eps"
                            # visualise training history
                            plt.plot(gru_training.history['acc'])
                            plt.plot(gru_training.history['val_acc'])
                            plt.title('model accuracy')
                            plt.ylabel('accuracy')
                            plt.xlabel('epoch')
                            #plt.legend(['gru_train', 'gru_test'], loc="lower right")
                            plt.savefig(img_gru, format='eps')
                            plt.show()


                            # create BiGRU
                            lr_sched_bigru = step_decay_schedule(initial_lr=1e-3, decay_factor=0.65, step_size=5)
                            red_lr  = ReduceLROnPlateau(monitor='val_loss', factor=0.4, patience=3, min_lr=0.0001)
                            earl_bigru = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=5)
                            #opt_bigru = tf.keras.optimizers.Adam(learning_rate=0.01)
                            opt_bigru = tf.keras.optimizers.Adam()
                            self.bidirect_gru = Sequential()
                            self.bidirect_gru.add(Embedding(input_dim   =  self.src_num_tokens,         # vocabulary size - number of unique words in data
                            output_dim    =  EMBEDDING_SIZE,          # length of vector with which each word is represented
                            input_length  =  self.MAX_SEQ_LENGTH,          # length of input sequence
                            weights       = [self.code_switch_embedding_matrix],      # word embedding matrix
                            trainable     =  True                     # True - update the embeddings while training
                            ))
                            self.bidirect_gru.add(Bidirectional(GRU(64, return_sequences=True)))
                            self.bidirect_gru.add(Dense(32, activation='relu'))
                            self.bidirect_gru.add(TimeDistributed(Dense(self.NUM_CLASSES, activation='softmax')))
                            self.bidirect_gru.compile(loss='categorical_crossentropy',
                                        optimizer=opt_bigru,
                                        metrics=['acc'])
                            print('\n','************* TRAINING BIDIRECTIONAL GRU *************','\n')
                            bidigru_training = self.bidirect_gru.fit(self.X_train, self.Y_train, batch_size=batch_s, epochs=epochs, validation_data=(self.X_validation, self.Y_validation),callbacks=[f1_callback, lr_sched_bigru,red_lr], verbose = 2)
                            img_bigru = img_dir + "X_bigru_acc_plot_" +source + '-' + target_lang + ".eps"
                            # visualise training history
                            plt.plot(bidigru_training.history['acc'])
                            plt.plot(bidigru_training.history['val_acc'])
                            plt.title('model accuracy')
                            plt.ylabel('accuracy')
                            plt.xlabel('epoch')
                            #plt.legend(['gru_train', 'gru_test'], loc="lower right")
                            plt.savefig(img_bigru, format='eps')
                            plt.show()
                            '''

                            # create BiLSTM architecture
                            lr_sched = step_decay_schedule(initial_lr=1e-5, decay_factor=0.3, step_size=9)
                            earl_bilstm = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=3)
                            opt_bilstm = tf.keras.optimizers.Adam()
                            self.bidirect_model = Sequential()
                            self.bidirect_model.add(Embedding(input_dim   =  self.src_num_tokens,         # vocabulary size - number of unique words in data
                            output_dim    =  EMBEDDING_SIZE,          # length of vector with which each word is represented
                            input_length  =  self.MAX_SEQ_LENGTH,          # length of input sequence
                            # weights       = [self.code_switch_embedding_matrix],      # word embedding matrix
                            weights       = [self.trg_embedding_matrix],
                            trainable     =  True                     # True - update the embeddings while training
                            ))
                            self.bidirect_model.add(Bidirectional(LSTM(64, return_sequences=True)))
                            self.bidirect_model.add(Dense(32, activation='relu'))
                            self.bidirect_model.add(TimeDistributed(Dense(self.NUM_CLASSES, activation='softmax')))
                            self.bidirect_model.compile(loss='categorical_crossentropy',
                                        optimizer=opt_bilstm,
                                        metrics=['acc'])
                            print('\n','************* TRAINING BIDIRECTIONAL LSTM *************','\n')
                            bidirect_training = self.bidirect_model.fit(self.X_train, self.Y_train, batch_size=batch_s, epochs=epochs, validation_data=(self.X_validation, self.Y_validation),callbacks=[f1_callback, lr_sched,earl_bilstm], verbose = 2)

                            img_bilstm = img_dir + "X_bilstm_acc_plot_" + source + '-' + target_lang + ".eps"
                            # visualise training history
                            plt.plot(bidirect_training.history['acc'])
                            plt.plot(bidirect_training.history['val_acc'])
                            plt.title('model accuracy')
                            plt.ylabel('accuracy')
                            plt.xlabel('epoch')
                            plt.legend(['rnn_train', 'rnn_test', 'lstm_train', 'lstm_test','gru_train', 'gru_test', 'bigru_train', 'bigru_test','bils_train', 'bils_test'], loc="lower right")
                            plt.savefig(img_bilstm, format='eps')
                            plt.show()



                            with open(os.path.join(destination, embed_type + '_' + type +'_results_f1'+ source+ "-" + target_lang + 'run1' + '.txt'),'w') as writer:
                                            '''
                                            loss, accuracy = self.rnn_model.evaluate(self.X_test, self.Y_test, verbose = 1)
                                            f, r, p = f1_rec_prec(self.rnn_model, self.X_test, self.Y_test)
                                            writer.write('RNN')
                                            writer.write('\n')
                                            writer.write("Loss: {0},\nAccuracy: {1}, \nF1 Score: {2},  \nRecall: {3},  \nPrecision: {4} ".format(loss, accuracy, f, r, p))
                                            writer.write('\n\n')


                                            loss, accuracy = self.lstm_model.evaluate(self.X_test, self.Y_test, verbose = 1)
                                            f, r, p = f1_rec_prec(self.lstm_model, self.X_test, self.Y_test)
                                            writer.write('LSTM')
                                            writer.write('\n')
                                            writer.write("Loss: {0},\nAccuracy: {1}, \nF1 Score: {2},  \nRecall: {3},  \nPrecision: {4} ".format(loss, accuracy, f, r, p))
                                            writer.write('\n\n')

                                            loss, accuracy = self.gru_model.evaluate(self.X_test, self.Y_test, verbose = 1)
                                            f, r, p = f1_rec_prec(self.gru_model, self.X_test, self.Y_test)
                                            writer.write('GRU')
                                            writer.write('\n')
                                            writer.write("Loss: {0},\nAccuracy: {1}, \nF1 Score: {2},  \nRecall: {3},  \nPrecision: {4} ".format(loss, accuracy, f, r, p))
                                            writer.write('\n\n')

                                            loss, accuracy = self.bidirect_gru.evaluate(self.X_test, self.Y_test, verbose = 1)
                                            f, r, p = f1_rec_prec(self.bidirect_gru, self.X_test, self.Y_test)
                                            writer.write('Bidirectional GRU')
                                            writer.write('\n')
                                            writer.write("Loss: {0},\nAccuracy: {1}, \nF1 Score: {2},  \nRecall: {3},  \nPrecision: {4} ".format(loss, accuracy, f, r, p))
                                            writer.write('\n\n')
                                            '''


                                            loss, accuracy = self.bidirect_model.evaluate(self.X_test, self.Y_test, verbose = 1)
                                            f, r, p = f1_rec_prec(self.bidirect_model, self.X_test, self.Y_test)
                                            writer.write('Bidirectional LSTM')
                                            writer.write('\n')
                                            writer.write("Loss: {0},\nAccuracy: {1}, \nF1 Score: {2},  \nRecall: {3},  \nPrecision: {4} ".format(loss, accuracy, f, r, p))
                                            writer.write('\n\n')



# Visualize Cross-lingual Embeddings
* This takes a bit of time to align the embeddings

In [ ]:
import os, sys
import time
modules_path = os.path.join('../src')
sys.path.append(modules_path)
from crosslingual import align_lex, plot_bilex, pca_bilex, tsne_plot
import pickle
# plot_bilex, pca_bilex

In [ ]:
source_lang = ['en'] #here you would list all 11 languages
target_lang = ['xho'] #here you would list all 11 languages
path_in = '../ext_data/M2M_MT_train_Embeddings/en_xho'
path_lex = './bilexicons/en_xho'
path_out = './processed/VecMap' #where aligned lexicon data will be stored and read from again

In [ ]:
# align vectors
for o in source_lang:
    for p in target_lang:
        if p != o:
            file_source = o+'_'+p+'-src-vcmp.txt'
            file_target = o+'_'+p+'-trg-vcmp.txt'
            file_source = os.path.join(path_in,file_source)
            file_target = os.path.join(path_in,file_target)
            bilex = o+'-'+p+'-bilex.txt'
            bilex = os.path.join(path_lex,bilex)

            align_lex(file_source, file_target, bilex, path_out, o, p, 'vecmap') # specificy if it is cca or vecmap, since cca has no header in txt file whereas vecmap does



In [ ]:
source_lang = ['en'] #here you would list all 11 languages
target_lang = ['xho'] #here you would list all 11 languages
path_in = './processed/VecMap'
path_out = './figures'   #where aligned lexicon data will be stored and read from again

In [ ]:
for o in source_lang:
    for p in target_lang:
        if p != o:
            print('Processing ********     ',o,' - to - ',p)
            l1 = o
            l2 = p
            name1=l1+'2'+l2+'_'+l1+'vec.txt'
            name2=l1+'2'+l2+'_'+l2+'vec.txt'

            name1 = os.path.join(path_in,name1)
            name2 = os.path.join(path_in,name2)

            with open(name1, 'rb') as fp:
                vecs=pickle.load(fp)
            with open(name2, 'rb') as fp:
                vectar=pickle.load(fp)

            name1=l1+'2'+l2+'_'+l1+'.txt'
            name2=l1+'2'+l2+'_'+l2+'.txt'

            name1 = os.path.join(path_in,name1)
            name2 = os.path.join(path_in,name2)

            with open(name1, 'rb') as fp:
                source1=pickle.load(fp)
            with open(name2, 'rb') as fp:
                target1=pickle.load(fp)

            fig_name = l1+'2'+l2+' ('+str(start)+'-'+str(end)+').png'
            fig_name = os.path.join(path_out,fig_name)

            a1,b1,c1,d1 = plot_bilex(source1, target1, vecs, vectar, fig_name, start=start,end=end)

            #while True:
            #    try:
            #        a1,b1,c1,d1 = plot_bilex(source1, target1, vecs, vectar, fig_name, start=start,end=end)
            #    except ValueError:
            #        print('Redoing ******** ',o,' - to - ',p)
                    #a1,b1,c1,d1 = plot_bilex(source1, target1, vecs, vectar, fig_name, start=start,end=end)
                #else:
                #    break

            name1=l1+'2'+l2+'_cos.txt'
            name1 = os.path.join(path_in,name1)

            with open(name1, 'wb') as fp:
                source1=pickle.dump(d1,fp)

# Cross-lingual Training

In [ ]:
# Languages = ["nso", "tsn", "xho", "zul", "sot"]

In [ ]:
# Choose languages
param1 = "en"
param2 = "xho"

data_path = "./data/xho"
cross_emb_path = "../ext_data/M2M_MT_train_Embeddings"                       # Path to embeddings
mono_path      = "../../../../"
model_destination_path = "/../../../../ext_data/thapelo/"                                         # path to output directory
img_dir = "../ext_data/"                               # path to plots
output_text = "./evaluation_crosslingual.txt"

# Train params
epochs = 100
batch_s = 418
emb_size = 200
type = 'vecmap'
embed_type = 'crosslingual'
destination_reports = "../ext_data/"
confus_path = "../ext_data/"

In [ ]:
# Start training.
if __name__=='__main__':
            class_obj = Data_Model_Wrapper_Code_Switch(data_path, mono_path, cross_emb_path, model_destination_path)
            class_obj.load_train_dev_test()
            class_obj.get_emb_mat(param1, param2)
            class_obj.seq_to_seq_models(epochs, batch_s, type, destination_reports, embed_type, param1, param2, emb_size)

            inference(class_obj.bidirect_model, class_obj.X_test,  class_obj.tag_tokenizer,class_obj.word_tokenizer,output_text )
            #confusion_m_plot(class_obj.bidirect_model, class_obj.X_test, class_obj.Y_test, param1 + '-' + param2, class_obj.tag_tokenizer, 'bi_lstm')
            #confusion_m_plot(class_obj.lstm_model, class_obj.X_test, class_obj.Y_test, param1 + '-' + param2, class_obj.tag_tokenizer, 'lstm')
            #confusion_m_plot(class_obj.gru_model, class_obj.X_test, class_obj.Y_test, param1 + '-' + param2, class_obj.tag_tokenizer, 'gru')
            #confusion_m_plot(class_obj.bidirect_gru, class_obj.X_test, class_obj.Y_test, param1 + '-' + param2, class_obj.tag_tokenizer, 'bi_gru')
            #confusion_m_plot(class_obj.rnn_model, class_obj.X_test, class_obj.Y_test, param1 + '-' + param2, class_obj.tag_tokenizer, 'rnn')
"""# Monolingual embeddings Training"""

# Model evaluation

# Monolingual embeddings Training

In [ ]:
class Data_Model_Wrapper_Monolingual:
        def __init__(self,root_data, root_mono, root_cross, path_to_destination_eda):
                    # initialize paths
#                     self.source_data_p = root_data + "/"
#                     self.target_data_p = root_data + "/"
                    self.data_path     = root_data
                    self.path_to_mono_emb = root_mono + "/"
                    self.path_to_cross_emb = root_cross + "/"
                    self.path_to_eda_output = path_to_destination_eda + "/"

        # def merge the two emb
        def code_switch_emb(self, dict1, dict2):
                        combined_dict = dict1.copy()
                        for key, value in dict2.items():
                            if key in combined_dict:
                                # Calculate the average of lists if the key is present in both dictionaries
                                combined_dict[key] = [(a + b) / 2 for a, b in zip(combined_dict[key], value)]
                            else:
                                # Add the key-value pair if it's not present in the first dictionary
                                combined_dict[key] = value

                        return combined_dict

        # load embeddings
        def get_emb_mat(self, src_language, trg_language):
                    # loading cross_lingual embeddings

                    directory_trg = self.path_to_mono_emb + 	 trg_language + "/"
                    file_names_trg = [f for f in os.listdir(directory_trg) if os.path.isfile(os.path.join(directory_trg, f))]
                    directory_src = self.path_to_mono_emb +  src_language + "/"
                    file_names_src = [f for f in os.listdir(directory_src) if os.path.isfile(os.path.join(directory_src, f))]

                    src_x_emb_file_p = directory_src + file_names_src[0]
                    trg_x_emb_file_p =  directory_trg   + file_names_trg[0]


                    # load src cross emb
                    # src_embeddings_index = {}
                    # with open(src_x_emb_file_p) as f:
                    #                 for  line in f:
                    #                         word, coefs = line.split(maxsplit=1)
                    #                         coefs = np.fromstring(coefs, "f", sep=" ")
                    #                         src_embeddings_index[word] = coefs

                    # load target cross embeddings
                    trg_embeddings_index = {}
                    with open(trg_x_emb_file_p) as f:
                                    for  line in f:
                                            word, coefs = line.split(maxsplit=1)
                                            coefs = np.fromstring(coefs, "f", sep=" ")
                                            trg_embeddings_index[word] = coefs


                    src_vocab = self.word_tokenizer.word_index
                    self.src_num_tokens = len(self.word_tokenizer.word_index) + 1
                    #print(src_num_tokens)
                    embedding_dim =  200
                    hits_src = 0
                    misses_src = 0
                    hits_trg = 0
                    misses_trg = 0
                    input_size = 200
                    self.code_switch_embedding_matrix = np.zeros((self.src_num_tokens, embedding_dim))
                    # for word, i in word2idx_all.items():
                    self.trg_embedding_matrix = np.zeros((self.src_num_tokens, embedding_dim))
                    # embeddings_index    =  self.code_switch_emb(src_embeddings_index, trg_embeddings_index)
                    for word in list(src_vocab.keys()):
                                    # trg emb
                                    embedding_vector = trg_embeddings_index.get(word)
                                    if embedding_vector is not None:
                                        self.trg_embedding_matrix[src_vocab[word]] = embedding_vector
                                        # self.code_switch_embedding_matrix[src_vocab[word]] = embedding_vector
                                        hits_trg +=1
                                    else:

                                        misses_trg += 1
#                     self.code_switch_layer = Embedding(self.src_num_tokens, embedding_dim, input_length=20, embeddings_initializer = keras.initializers.Constant(self.code_switch_embedding_matrix))
                    #print(self.code_switch_layer)


    #                   return code_switch_layer

        # load data
        def load_train_dev_test(self):
                            # load datasets
                            try:
                                train = open(os.path.join(self.data_path ,'train.txt'),'r').readlines()
                            except FileNotFoundError:
                                train = ''
                            try:
                                dev = open(os.path.join(self.data_path ,'dev.txt'),'r').readlines()
                            except FileNotFoundError:
                                dev = ''

                            try:
                                test = open(os.path.join(self.data_path ,'test.txt'),'r').readlines()
                            except FileNotFoundError:
                                test = ''



                            new = []
                            sent = []
                            tag = []
                            X = []
                            Y = []
                            for t in train:
                                if t.strip() != '':
                                    split = t.strip().split(' ')
                                    if len(split) > 1:
                                        #print(split)
                                        sent.append(split[0])
                                        tag.append(split[1])

                                else:
                                    new_s = ' '.join(sent)
                                    new_t = ' '.join(tag)
                                    X.append(sent)
                                    Y.append(tag)
                                    new.append([new_s,new_t])
                                    sent = []
                                    tag = []

                            sent = []
                            tag = []

                            if dev != '':
                                for d in dev:
                                        if d.strip() != '':
                                            split = d.strip().split(' ')
                                            sent.append(split[0])
                                            tag.append(split[1])
                                        else:
                                            new_s = ' '.join(sent)
                                            new_t = ' '.join(tag)
                                            X.append(sent)
                                            Y.append(tag)
                                            new.append([new_s,new_t])
                                            sent = []
                                            tag = []

                            sent = []
                            tag = []

                            if test != '':
                                for tt in test:
                                    if tt.strip() != '':
                                        split = tt.strip().split(' ')
                                        sent.append(split[0])
                                        tag.append(split[1])
                                    else:
                                        new_s = ' '.join(sent)
                                        new_t = ' '.join(tag)
                                        X.append(sent)
                                        Y.append(tag)
                                        new.append([new_s,new_t])
                                        sent = []
                                        tag = []


                            num_words = len(set([word.lower() for sentence in X for word in sentence]))
                            num_tags   = len(set([word.lower() for sentence in Y for word in sentence]))
                            print("Total number of tagged sentences: {}".format(len(X)))
                            print("Vocabulary size: {}".format(num_words))
                            print("Total number of tags: {}".format(num_tags))



                            self.word_tokenizer = Tokenizer()                      # instantiate tokeniser
                            self.word_tokenizer.fit_on_texts(X)                    # fit tokeniser on data
                            X_encoded = self.word_tokenizer.texts_to_sequences(X)  # use the tokeniser to encode input sequence

                            # encode Y
                            self.tag_tokenizer = Tokenizer()
                            self.tag_tokenizer.fit_on_texts(Y)
                            Y_encoded = self.tag_tokenizer.texts_to_sequences(Y)

                            # make sure that each sequence of input and output is same length
                            different_length = [1 if len(input) != len(output) else 0 for input, output in zip(X_encoded, Y_encoded)]
                            print("{} sentences have disparate input-output lengths.".format(sum(different_length)))

                            lengths = [len(seq) for seq in X_encoded]
                            self.MAX_SEQ_LENGTH = max(lengths)  # sequences greater than 100 in length will be truncated
                            X_padded = pad_sequences(X_encoded, maxlen=self.MAX_SEQ_LENGTH, padding="pre", truncating="post")
                            Y_padded = pad_sequences(Y_encoded, maxlen=self.MAX_SEQ_LENGTH, padding="pre", truncating="post")

                            # assign padded sequences to X and Y
                            X, Y = X_padded, Y_padded
                            Y = to_categorical(Y)
                            Y = [array[:, 1:] for array in Y]
                            # Convert the result back to a list of NumPy arrays
                            Y = np.array([np.array(arr) for arr in Y])

                            x_l, d_l = len(train), len(dev)

                            self.X_train       =  X[:x_l]
                            self.Y_train       =  Y[:x_l]
                            self.X_test        =  X[x_l + d_l: ]
                            self.Y_test        =  Y[x_l + d_l: ]

                            self.X_validation  =   X[x_l: x_l + d_l]
                            self.Y_validation  =   Y[x_l: x_l + d_l]


                            # print number of samples in each set
                            print("TRAINING DATA")
                            print('Shape of input sequences: {}'.format(self.X_train.shape))
                            print('Shape of output sequences: {}'.format(self.Y_train.shape))
                            print("-"*50)
                            print("VALIDATION DATA")
                            print('Shape of input sequences: {}'.format(self.X_validation.shape))
                            print('Shape of output sequences: {}'.format(self.Y_validation.shape))
                            print("-"*50)
                            print("TESTING DATA")
                            print('Shape of input sequences: {}'.format(self.X_test.shape))
                            print('Shape of output sequences: {}'.format(self.Y_test.shape))
                            self.NUM_CLASSES = Y.shape[2]
                            #print(Y.shape)
    #                       return X_train, Y_train, X_validation, Y_validation, X_test,Y_test


        # training models
        def seq_to_seq_models(self,epochs, batch_s, type, destination, embed_type, source, target_lang, EMBEDDING_SIZE):

                            # Assuming you have self.X_validation and self.Y_validation for validation data
                            f1_callback = F1ScoreCallback(self.X_train, self.Y_train, self.X_validation, self.Y_validation)
                            '''
                            #lr_sched_rn = step_decay_schedule(initial_lr=1e-5, decay_factor=0.38, step_size=2)
                            # Define a callback to modify the learning rate dynamically
                            earl_rnn = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=10)
                            reduce_lr_rnn = ReduceLROnPlateau(monitor='val_loss', factor=0.14, patience=3, min_lr=0.000001)
                            # create RNN architecture
                            opt_rnn = tf.keras.optimizers.Adam(learning_rate=0.0001)
                            self.rnn_model = Sequential()
                            # create embedding layer - usually the first layer in text problems
                            self.rnn_model.add(Embedding(input_dim   =  self.src_num_tokens,         # vocabulary size - number of unique words in data
                            output_dim    =  EMBEDDING_SIZE,          # length of vector with which each word is represented
                            input_length  =  self.MAX_SEQ_LENGTH,          # length of input sequence
                            weights       = [self.code_switch_embedding_matrix],      # word embedding matrix
                            trainable     =  True                     # True - update the embeddings while training
                            ))
                            # add an RNN layer which contains 64 RNN cells
                            self.rnn_model.add(SimpleRNN(64,
                                        return_sequences=True  # True - return whole sequence; False - return single output of the end of the sequence
                            ))

                            # add time distributed (output at each sequence) layer
                            self.rnn_model.add(Dense(32, activation='relu'))
                            self.rnn_model.add(TimeDistributed(Dense(self.NUM_CLASSES, activation='softmax')))
                            self.rnn_model.compile(loss      =  'categorical_crossentropy',
                                            optimizer =  opt_rnn,
                                            metrics   =  ['acc'])
                            print('\n','************* TRAINING RNN *************','\n')
                            rnn_training = self.rnn_model.fit(self.X_train, self.Y_train, batch_size=282, epochs=epochs, validation_data=(self.X_validation, self.Y_validation), callbacks=[f1_callback, earl_rnn, reduce_lr_rnn], verbose = 2)
                            img_rnn =  img_dir + "X_rnn_acc_plot_" +source + '-' + target_lang + ".eps"
                            # visualise training history
                            plt.plot(rnn_training.history['acc'])
                            plt.plot(rnn_training.history['val_acc'])
                            plt.title('model accuracy')
                            plt.ylabel('accuracy')
                            plt.xlabel('epoch')
                            #plt.legend(['rnn_train', 'rnn_test'], loc="lower right")
                            plt.savefig(img_rnn, format='eps')
                            plt.show()


                            #lr_sched_lstm = step_decay_schedule(initial_lr=1e-3, decay_factor=0.75, step_size=6)
                            earl_lstm = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=10)
                            # create LSTM architecture10
                            opt_lstm = tf.keras.optimizers.Adam(learning_rate=0.0001)
                            reduce_lr_lstm = ReduceLROnPlateau(monitor='val_loss', factor=0.68, patience=6, min_lr=0.000001)
                            #opt_lstm = tf.keras.optimizers.Adam()
                            self.lstm_model = Sequential()
                            self.lstm_model.add(Embedding(input_dim   =  self.src_num_tokens,         # vocabulary size - number of unique words in data
                            output_dim    =  EMBEDDING_SIZE,          # length of vector with which each word is represented
                            input_length  =  self.MAX_SEQ_LENGTH,          # length of input sequence
                            weights       = [self.code_switch_embedding_matrix],      # word embedding matrix
                            trainable     =  True                     # True - update the embeddings while training
                            ))
                            self.lstm_model.add(LSTM(64, return_sequences=True))
                            self.lstm_model.add(Dense(32, activation='relu'))
                            self.lstm_model.add(TimeDistributed(Dense(self.NUM_CLASSES, activation='softmax')))
                            self.lstm_model.compile(loss      =  'categorical_crossentropy',
                                            optimizer =  opt_lstm,
                                            metrics   =  ['acc'])
                            print('\n','************* TRAINING LSTM *************','\n')
                            lstm_training = self.lstm_model.fit(self.X_train, self.Y_train, batch_size=batch_s, epochs=epochs, validation_data=(self.X_validation, self.Y_validation), callbacks=[f1_callback,  reduce_lr_lstm, earl_lstm], verbose = 2)
                            img_lstm = img_dir + "X_lstm_acc_plot_" +source + '-' + target_lang + ".eps"
                            # visualise training history
                            plt.plot(lstm_training.history['acc'])
                            plt.plot(lstm_training.history['val_acc'])
                            plt.title('model accuracy')
                            plt.ylabel('accuracy')
                            plt.xlabel('epoch')
                            #plt.legend(['lstm_train', 'lstm_test'], loc="lower right")
                            plt.savefig(img_lstm, format='eps')
                            plt.show()


                            # create GRU architecture
                            lr_sched_gru = step_decay_schedule(initial_lr=1e-5, decay_factor=0.1, step_size=8)
                            earl_gru = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=10)
                            #opt_gru = tf.keras.optimizers.Adam(learning_rate=0.001)
                            opt_gru = tf.keras.optimizers.Adam()
                            self.gru_model = Sequential()
                            self.gru_model.add(Embedding(input_dim   =  self.src_num_tokens,         # vocabulary size - number of unique words in data
                            output_dim    =  EMBEDDING_SIZE,          # length of vector with which each word is represented
                            input_length  =  self.MAX_SEQ_LENGTH,          # length of input sequence
                            weights       = [self.code_switch_embedding_matrix],      # word embedding matrix
                            trainable     =  True                     # True - update the embeddings while training
                            ))
                            self.gru_model.add(GRU(64, return_sequences=True))
                            self.gru_model.add(Dense(32, activation='relu'))
                            self.gru_model.add(TimeDistributed(Dense(self.NUM_CLASSES, activation='softmax')))
                            self.gru_model.compile(loss='categorical_crossentropy',
                                        optimizer= opt_gru,
                                        metrics=['acc'])
                            print('\n','************* TRAINING GRU *************','\n')
                            gru_training = self.gru_model.fit(self.X_train, self.Y_train, batch_size=batch_s, epochs=epochs, validation_data=(self.X_validation, self.Y_validation), callbacks=[f1_callback, earl_gru, lr_sched_gru], verbose = 2 )
                            img_gru = img_dir + "X_gru_acc_plot_" +source + '-' + target_lang + ".eps"
                            # visualise training history
                            plt.plot(gru_training.history['acc'])
                            plt.plot(gru_training.history['val_acc'])
                            plt.title('model accuracy')
                            plt.ylabel('accuracy')
                            plt.xlabel('epoch')
                            #plt.legend(['gru_train', 'gru_test'], loc="lower right")
                            plt.savefig(img_gru, format='eps')
                            plt.show()

                            # create BiGRU
                            lr_sched_bigru = step_decay_schedule(initial_lr=1e-5, decay_factor=0.2, step_size=10)
                            earl_bigru = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=10)
                            #opt_bigru = tf.keras.optimizers.Adam(learning_rate=0.001)
                            opt_bigru = tf.keras.optimizers.Adam()
                            self.bidirect_gru = Sequential()
                            self.bidirect_gru.add(Embedding(input_dim   =  self.src_num_tokens,         # vocabulary size - number of unique words in data
                            output_dim    =  EMBEDDING_SIZE,          # length of vector with which each word is represented
                            input_length  =  self.MAX_SEQ_LENGTH,          # length of input sequence
                            weights       = [self.code_switch_embedding_matrix],      # word embedding matrix
                            trainable     =  True                     # True - update the embeddings while training
                            ))
                            self.bidirect_gru.add(Bidirectional(GRU(64, return_sequences=True)))
                            self.bidirect_gru.add(Dense(32, activation='relu'))
                            self.bidirect_gru.add(TimeDistributed(Dense(self.NUM_CLASSES, activation='softmax')))
                            self.bidirect_gru.compile(loss='categorical_crossentropy',
                                        optimizer=opt_bigru,
                                        metrics=['acc'])
                            print('\n','************* TRAINING BIDIRECTIONAL GRU *************','\n')
                            bidigru_training = self.bidirect_gru.fit(self.X_train, self.Y_train, batch_size=512, epochs=epochs, validation_data=(self.X_validation, self.Y_validation),callbacks=[f1_callback, earl_bigru,lr_sched_bigru], verbose = 2)
                            img_bigru = img_dir + "X_bigru_acc_plot_" +source + '-' + target_lang + ".eps"
                            # visualise training history
                            plt.plot(bidigru_training.history['acc'])
                            plt.plot(bidigru_training.history['val_acc'])
                            plt.title('model accuracy')
                            plt.ylabel('accuracy')
                            plt.xlabel('epoch')
                            #plt.legend(['gru_train', 'gru_test'], loc="lower right")
                            plt.savefig(img_bigru, format='eps')
                            plt.show()
                            '''

                            # create BiLSTM architecture
                            lr_sched = step_decay_schedule(initial_lr=1e-5, decay_factor=0.3, step_size=10)
                            earl_bilstm = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=10)
                            opt_bilstm = tf.keras.optimizers.Adam()
                            self.bidirect_model = Sequential()
                            self.bidirect_model.add(Embedding(input_dim   =  self.src_num_tokens,         # vocabulary size - number of unique words in data
                            output_dim    =  EMBEDDING_SIZE,          # length of vector with which each word is represented
                            input_length  =  self.MAX_SEQ_LENGTH,          # length of input sequence
                            weights       = [self.trg_embedding_matrix],      # word embedding matrix
                            trainable     =  True                     # True - update the embeddings while training
                            ))
                            self.bidirect_model.add(Bidirectional(LSTM(64, return_sequences=True)))
                            self.bidirect_model.add(Dense(32, activation='relu'))
                            self.bidirect_model.add(TimeDistributed(Dense(self.NUM_CLASSES, activation='softmax')))
                            self.bidirect_model.compile(loss='categorical_crossentropy',
                                        optimizer=opt_bilstm,
                                        metrics=['acc'])
                            print('\n','************* TRAINING BIDIRECTIONAL LSTM *************','\n')
                            bidirect_training = self.bidirect_model.fit(self.X_train, self.Y_train, batch_size=batch_s, epochs=epochs, validation_data=(self.X_validation, self.Y_validation),callbacks=[f1_callback, lr_sched,earl_bilstm], verbose = 2)

                            img_bilstm = img_dir + "X_bilstm_acc_plot_" + source + '-' + target_lang + ".eps"
                            # visualise training history
                            plt.plot(bidirect_training.history['acc'])
                            plt.plot(bidirect_training.history['val_acc'])
                            plt.title('model accuracy')
                            plt.ylabel('accuracy')
                            plt.xlabel('epoch')
                            plt.legend(['rnn_train', 'rnn_test', 'lstm_train', 'lstm_test','gru_train', 'gru_test', 'bigru_train', 'bigru_test','bils_train', 'bils_test'], loc="lower right")
                            plt.savefig(img_bilstm, format='eps')
                            plt.show()

                            with open(os.path.join(destination, embed_type + '_' + type +'_results_f1'+ source+ "-" + target_lang + 'run1' + '.txt'),'w') as writer:
                                            # loss, accuracy = self.rnn_model.evaluate(self.X_test, self.Y_test, verbose = 1)
                                            # f, r, p = f1_rec_prec(self.rnn_model, self.X_test, self.Y_test)
                                            # writer.write('RNN')
                                            # writer.write('\n')
                                            # writer.write("Loss: {0},\nAccuracy: {1}, \nF1 Score: {2},  \nRecall: {3},  \nPrecision: {4} ".format(loss, accuracy, f, r, p))
                                            # writer.write('\n\n')

                                            # loss, accuracy = self.lstm_model.evaluate(self.X_test, self.Y_test, verbose = 1)
                                            # f, r, p = f1_rec_prec(self.lstm_model, self.X_test, self.Y_test)
                                            # writer.write('LSTM')
                                            # writer.write('\n')
                                            # writer.write("Loss: {0},\nAccuracy: {1}, \nF1 Score: {2},  \nRecall: {3},  \nPrecision: {4} ".format(loss, accuracy, f, r, p))
                                            # writer.write('\n\n')

                                            # loss, accuracy = self.gru_model.evaluate(self.X_test, self.Y_test, verbose = 1)
                                            # f, r, p = f1_rec_prec(self.gru_model, self.X_test, self.Y_test)
                                            # writer.write('GRU')
                                            # writer.write('\n')
                                            # writer.write("Loss: {0},\nAccuracy: {1}, \nF1 Score: {2},  \nRecall: {3},  \nPrecision: {4} ".format(loss, accuracy, f, r, p))
                                            # writer.write('\n\n')

                                            # loss, accuracy = self.bidirect_gru.evaluate(self.X_test, self.Y_test, verbose = 1)
                                            # f, r, p = f1_rec_prec(self.bidirect_gru, self.X_test, self.Y_test)
                                            # writer.write('Bidirectional GRU')
                                            # writer.write('\n')
                                            # writer.write("Loss: {0},\nAccuracy: {1}, \nF1 Score: {2},  \nRecall: {3},  \nPrecision: {4} ".format(loss, accuracy, f, r, p))
                                            # writer.write('\n\n')

                                            loss, accuracy = self.bidirect_model.evaluate(self.X_test, self.Y_test, verbose = 1)
                                            f, r, p = f1_rec_prec(self.bidirect_model, self.X_test, self.Y_test)
                                            writer.write('Bidirectional LSTM')
                                            writer.write('\n')
                                            writer.write("Loss: {0},\nAccuracy: {1}, \nF1 Score: {2},  \nRecall: {3},  \nPrecision: {4} ".format(loss, accuracy, f, r, p))
                                            writer.write('\n\n')

In [ ]:
# Languages = ["nso", "tsn", "xho", "zul", "sot"]

In [ ]:
# Choose languages
param1 = "en"
param2 = "xho"

data_path = "./data/xho"
cross_emb_path = "../ext_data/M2M_MT_train_Embeddings"                       # Path to embeddings
mono_path      = "../ext_data/M2M_mono_Embeddings/m2m-mono-embeddings"
model_destination_path = "/../../../../ext_data/thapelo/"                                         # path to output directory
img_dir = "../ext_data/"                               # path to plots
# Train params
epochs = 100
batch_s = 418
emb_size = 200
type = 'vecmap'
embed_type = 'crosslingual'
destination_reports = "../ext_data/"
confus_path = "../ext_data/"
output_text = "./evaluation_monolingual.txt"

In [ ]:
# Start training
if __name__=='__main__':
            class_obj = Data_Model_Wrapper_Monolingual(data_path, mono_path, cross_emb_path, model_destination_path)
            class_obj.load_train_dev_test()
            class_obj.get_emb_mat(param1, param2)
            class_obj.seq_to_seq_models(epochs, batch_s, type, destination_reports, embed_type, param1, param2, emb_size)

            confusion_m_plot(class_obj.bidirect_model, class_obj.X_test, class_obj.Y_test, param1 + '-' + param2, class_obj.tag_tokenizer, 'bi_lstm')
            inference(class_obj.bidirect_model, class_obj.X_test, class_obj.tag_tokenizer,class_obj.word_tokenizer, output_text )

            # confusion_m_plot(class_obj.lstm_model, class_obj.X_test, class_obj.Y_test, param1 + '-' + param2, class_obj.tag_tokenizer, 'lstm')
            # confusion_m_plot(class_obj.gru_model, class_obj.X_test, class_obj.Y_test, param1 + '-' + param2, class_obj.tag_tokenizer, 'gru')
            # confusion_m_plot(class_obj.bidirect_gru, class_obj.X_test, class_obj.Y_test, param1 + '-' + param2, class_obj.tag_tokenizer, 'bi_gru')
            # confusion_m_plot(class_obj.rnn_model, class_obj.X_test, class_obj.Y_test, param1 + '-' + param2, class_obj.tag_tokenizer, 'rnn')

# Monolinual Plots